<h2 style="text-align: center;">
Exploring Export Financing Trends: A Data-Driven Analysis of FY25 Q3 Records
</h2>

**Step 2: Data pre‑processing**

**Goal**: Perform all necessary cleaning steps
Based on the issues identified in Step 1, we will now address and resolve them through data cleaning and preprocessing.


**Task to complete**:

2.1 Drop unnecessary columns

2.2 Rename Column Names

2.3 Handling missing values

2.4 Removing duplicates 

2.5 Correcting data types 

2.6 Creating derived columns 

2.7 Filtering or aggregating data 

2.8 Final Data Quality Check

2.9 Exporting Cleaned Dataset


**Importing Libraries**

We import pandas for data handling and numpy for numerical operations.  

Display settings are adjusted to show all columns and format numbers to two decimals.  

We can Confirm the Libraries were loaded successfully by the confirmation message.


In [1]:
#Importing Libraries
import pandas as pd
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)        # show all columns
pd.set_option('display.float_format', '{:.2f}'.format)  # format decimals to 2 places

print("Libraries imported successfully!")

Libraries imported successfully!


**Loading the Dataset**

The dataset is loaded directly from the official U.S. Export‑Import Bank (Data.gov). 

We can Confirm the dataset were loaded successfully by the confirmation message.

In [2]:
# Loading the Dataset
url = "https://img.exim.gov/s3fs-public/dataset/vbhv-d8am/Data.Gov_-_FY25_Q3.csv"
df = pd.read_csv(url, low_memory=False)

print("Dataset loaded successfully!")
print("="*70)
print(f"Dataset Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

Dataset loaded successfully!
Dataset Shape: 51,753 rows x 34 columns


**2.1 Drop unnecessary columns**

**Goal**: Remove identifiers and overly detailed fields that don’t add value to high‑level analysis.

In [3]:
columns_to_drop = [
    'Unique Identifier', 'Deal Number', 'Primary Applicant', 'Primary Lender',
    'Primary Borrower', 'Primary Exporter City', 'Primary Exporter State Code',
    'Primary Export Product NAICS/SIC code'
]

df.drop(columns=[c for c in columns_to_drop if c in df.columns], inplace=True)

print("Shape after dropping:", df.shape)
print("Remaining columns:\n", list(df.columns))


Shape after dropping: (51753, 26)
Remaining columns:
 ['Fiscal Year', 'Decision', 'Decision Date', 'Effective Date', 'Expiration Date', 'Brokered ', 'Deal Cancelled', 'Country', 'Program', 'Policy Type', 'Decision Authority', 'Product Description', 'Term', 'Primary Exporter', 'Primary Exporter State Name', 'Primary Source of Repayment (PSOR)', 'Working Capital Delegated Authority', 'Approved/Declined Amount', 'Disbursed/Shipped Amount', 'Undisbursed Exposure Amount', 'Outstanding Exposure Amount', 'Small Business Authorized Amount', 'Woman Owned Authorized Amount', 'Minority Owned Authorized Amount', 'Loan Interest Rate', 'Multiyear Working Capital Extension ']


**2.2 Rename column names**

**Goal**: Standardize headers to clean, consistent names.

In [4]:
rename_map = {
    'Fiscal Year': 'FiscalYear',
    'Decision': 'DecisionStatus',
    'Decision Date': 'DecisionDate',
    'Effective Date': 'EffectiveDate',
    'Expiration Date': 'ExpirationDate',
    'Brokered ': 'Brokered',
    'Deal Cancelled': 'DealCancelled',
    'Country': 'Country',
    'Program': 'ProgramType',
    'Policy Type': 'PolicyType',
    'Decision Authority': 'DecisionAuthority',
    'Product Description': 'ProductDescription',
    'Term': 'Term',
    'Primary Exporter': 'Exporter',
    'Primary Exporter State Name': 'ExporterState',
    'Primary Source of Repayment (PSOR)': 'PSOR',
    'Working Capital Delegated Authority': 'WCDelegatedAuthority',
    'Approved/Declined Amount': 'ApprovedAmount',
    'Disbursed/Shipped Amount': 'DisbursedAmount',
    'Undisbursed Exposure Amount': 'UndisbursedExposure',
    'Outstanding Exposure Amount': 'OutstandingExposure',
    'Small Business Authorized Amount': 'SmallBusinessAmount',
    'Woman Owned Authorized Amount': 'WomenOwnedAmount',
    'Minority Owned Authorized Amount': 'MinorityOwnedAmount',
    'Loan Interest Rate': 'InterestRate',
    'Multiyear Working Capital Extension ': 'MultiYearWCExtension'
}
df.rename(columns=rename_map, inplace=True)

print("Renamed columns:\n", list(df.columns))

Renamed columns:
 ['FiscalYear', 'DecisionStatus', 'DecisionDate', 'EffectiveDate', 'ExpirationDate', 'Brokered', 'DealCancelled', 'Country', 'ProgramType', 'PolicyType', 'DecisionAuthority', 'ProductDescription', 'Term', 'Exporter', 'ExporterState', 'PSOR', 'WCDelegatedAuthority', 'ApprovedAmount', 'DisbursedAmount', 'UndisbursedExposure', 'OutstandingExposure', 'SmallBusinessAmount', 'WomenOwnedAmount', 'MinorityOwnedAmount', 'InterestRate', 'MultiYearWCExtension']


**2.3 Handling missing values**

**Goal**: Summarize nulls, enforce presence of critical fields.

In this step, we resolved missing values by dropping rows with nulls in critical fields (DecisionStatus, Country, ProgramType, ApprovedAmount), filling numeric columns with their median values, replacing categorical gaps with "Unknown," and forward‑filling dates where appropriate to ensure the dataset remains consistent and analysis‑ready.

***Count missing values in each column***

In [5]:
# Count missing values in each column
missing_summary = df.isnull().sum()

print("Missing values per column:\n", missing_summary)

Missing values per column:
 FiscalYear                  0
DecisionStatus              0
DecisionDate                0
EffectiveDate            1241
ExpirationDate            686
Brokered                    0
DealCancelled               0
Country                     0
ProgramType                 0
PolicyType               9065
DecisionAuthority           7
ProductDescription       1496
Term                        0
Exporter                  191
ExporterState            1692
PSOR                        7
WCDelegatedAuthority    45692
ApprovedAmount              0
DisbursedAmount             0
UndisbursedExposure         0
OutstandingExposure         0
SmallBusinessAmount         0
WomenOwnedAmount            0
MinorityOwnedAmount         0
InterestRate            51381
MultiYearWCExtension    51502
dtype: int64


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51753 entries, 0 to 51752
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   FiscalYear            51753 non-null  int64  
 1   DecisionStatus        51753 non-null  object 
 2   DecisionDate          51753 non-null  object 
 3   EffectiveDate         50512 non-null  object 
 4   ExpirationDate        51067 non-null  object 
 5   Brokered              51753 non-null  object 
 6   DealCancelled         51753 non-null  object 
 7   Country               51753 non-null  object 
 8   ProgramType           51753 non-null  object 
 9   PolicyType            42688 non-null  object 
 10  DecisionAuthority     51746 non-null  object 
 11  ProductDescription    50257 non-null  object 
 12  Term                  51753 non-null  object 
 13  Exporter              51562 non-null  object 
 14  ExporterState         50061 non-null  object 
 15  PSOR               

In [7]:
df.isnull().values.any()

np.True_

**2.3.1. Drop rows with missing values in critical columns**

In [8]:
# Define critical columns
critical_cols = ['DecisionStatus', 'Country', 'ProgramType', 'ApprovedAmount']

# Drop rows where any of these are missing
df = df.dropna(subset=[c for c in critical_cols if c in df.columns])

print("Shape after dropping critical missing values:", df.shape)

Shape after dropping critical missing values: (51753, 26)


**2.3.2. Fill missing numeric values with median**

In [9]:
import numpy as np
import pandas as pd

# Define numeric columns
numeric_cols = ['ApprovedAmount', 'DisbursedAmount', 'OutstandingExposure', 'InterestRate']

# Fill missing values with median (only if median is valid)
for col in numeric_cols:
    if col in df.columns:
        try:
            # Convert to numeric safely
            df[col] = pd.to_numeric(df[col], errors='coerce')
            median_value = df[col].median()
            if not np.isnan(median_value):
                df[col] = df[col].fillna(median_value)
                print(f"Filled missing values in '{col}' with median: {median_value}")
            else:
                print(f"Skipped '{col}' — median is NaN (all values missing or non-numeric).")
        except Exception as e:
            print(f"Error in column '{col}': {e}")

Filled missing values in 'ApprovedAmount' with median: 500000.0
Filled missing values in 'DisbursedAmount' with median: 300000.0
Filled missing values in 'OutstandingExposure' with median: 0.0
Filled missing values in 'InterestRate' with median: 3.13


In Step 2.3.2, we handled missing values in numeric columns by:
- Converting each column to numeric using `pd.to_numeric()` to avoid type errors.
- Computing the median only if valid (not NaN).
- Filling missing values with the column’s median.
- Skipping columns where the median could not be computed due to all missing or invalid values.

**2.3.3. Fill missing categorical values with "Unknown"**

In [10]:
fill_map = {col: 'Unknown' for col in ['ProgramType', 'PolicyType', 'ExporterState'] if col in df.columns}
df = df.fillna(value=fill_map)

# Verify
print("Missing (after dict fill):")
print(df[list(fill_map.keys())].isnull().sum())

Missing (after dict fill):
ProgramType      0
PolicyType       0
ExporterState    0
dtype: int64


To handle missing values in categorical columns, we use a dictionary-based one-liner. This approach is clean, scalable, and avoids chained assignment warnings.

**2.3.4. Fill missing dates**

In [11]:
# Forward fill missing DecisionDate safely
if 'DecisionDate' in df.columns:
    df['DecisionDate'] = df['DecisionDate'].ffill()

# Output check
print("Missing values in DecisionDate after fill:", df['DecisionDate'].isnull().sum())

Missing values in DecisionDate after fill: 0


To handle missing values in the DecisionDate column, we used `.ffill()` to forward-fill gaps based on the previous valid date. This avoids chained assignment and deprecated method warnings, ensuring compatibility with future versions of pandas.

In [12]:
print("Remaining missing values:\n", df.isnull().sum().sort_values(ascending=False).head(10))

Remaining missing values:
 MultiYearWCExtension    51502
WCDelegatedAuthority    45692
ProductDescription       1496
EffectiveDate            1241
ExpirationDate            686
Exporter                  191
DecisionAuthority           7
PSOR                        7
DecisionStatus              0
FiscalYear                  0
dtype: int64


This shows the top 10 columns with the most missing values still present in the DataFrame.these columns still contain missing values.

We handled the remaining missing values by:
- Dropping columns with more missing data (e.g., MultiYearKExtension, WCDelegatedAuthority).
- Filling categorical gaps with `"Unknown"` in key fields like Exporter and ProductDescription.
- Forward-filling missing dates in EffectiveDate and ExpirationDate.
- Verifying that remaining missing values are minimal and non-critical.

In [13]:
# Step 2.3.5 — Handle remaining missing values based on severity

# 1. Show top 10 columns with missing values
missing_summary = df.isnull().sum().sort_values(ascending=False).head(10)
print("Top 10 columns with missing values:\n", missing_summary)

# 2. Drop columns with excessive missing values (e.g., > 90%)
high_null_cols = [col for col in df.columns if df[col].isnull().mean() > 0.9]
df.drop(columns=high_null_cols, inplace=True)
print("\nDropped columns with >90% missing values:", high_null_cols)

# 3. Fill remaining categorical columns with 'Unknown'
cat_fill_cols = ['Exporter', 'ProductDescription', 'PolicyType', 'ProgramType', 'ExporterState']
for col in cat_fill_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# 4. Fill remaining date columns with forward fill
date_fill_cols = ['EffectiveDate', 'ExpirationDate']
for col in date_fill_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').ffill()

# 5. Final check
print("\nRemaining missing values after cleanup:\n", df.isnull().sum().sort_values(ascending=False).head(10))

Top 10 columns with missing values:
 MultiYearWCExtension    51502
WCDelegatedAuthority    45692
ProductDescription       1496
EffectiveDate            1241
ExpirationDate            686
Exporter                  191
DecisionAuthority           7
PSOR                        7
DecisionStatus              0
FiscalYear                  0
dtype: int64

Dropped columns with >90% missing values: ['MultiYearWCExtension']

Remaining missing values after cleanup:
 WCDelegatedAuthority    45692
DecisionAuthority           7
PSOR                        7
EffectiveDate               1
FiscalYear                  0
Brokered                    0
DecisionStatus              0
DecisionDate                0
ExpirationDate              0
ProgramType                 0
dtype: int64


We completed the missing value cleanup by:
- Dropping `WCDelegatedAuthority` due to excessive nulls.
- Filling remaining categorical gaps with `"Unknown"` in columns like Lender, ExporterCity, and ProductCode.
- Dropping rows missing DealNumber to ensure record integrity.
- Verifying that all remaining missing values are resolved or non-critical.

In [14]:
# 1. Drop WCDelegatedAuthority (too sparse)
df.drop(columns=['WCDelegatedAuthority'], inplace=True)

# 2. Fill remaining categorical columns with 'Unknown'
fill_unknown_cols = ['Lender', 'ExporterCity', 'ExporterStateCode', 'ProductCode', 'Applicant', 'PSOR', 'DecisionAuthority']
for col in fill_unknown_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# 3. Drop rows with missing DealNumber (if still present)
if 'DealNumber' in df.columns:
    df = df.dropna(subset=['DealNumber'])

# 4. Final check
print(" Final missing values:\n", df.isnull().sum().sort_values(ascending=False).head(10))

 Final missing values:
 EffectiveDate     1
FiscalYear        0
DecisionStatus    0
DecisionDate      0
ExpirationDate    0
Brokered          0
DealCancelled     0
Country           0
ProgramType       0
PolicyType        0
dtype: int64


Our final check shows that EffectiveDate still has 1 missing value:
After forward-filling `EffectiveDate`, one missing value remained at the top of the dataset.  
We resolved it using `.bfill()` to backfill from the next available date, ensuring no gaps remain.

In [15]:
# Backfill the remaining missing EffectiveDate
if 'EffectiveDate' in df.columns:
    df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'], errors='coerce').bfill()

# Confirm it's resolved
print(" Final missing values:\n", df.isnull().sum().sort_values(ascending=False).head(10))

 Final missing values:
 FiscalYear        0
DecisionStatus    0
DecisionDate      0
EffectiveDate     0
ExpirationDate    0
Brokered          0
DealCancelled     0
Country           0
ProgramType       0
PolicyType        0
dtype: int64


Now our dataset is **cleaned and free of missing values**.

**2.4 Removing duplicates**


**2.4.1 Check how many duplicate rows exist and Preview a few duplicate rows (if any)**

In [16]:
duplicate_count = df.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)
print("Sample duplicates:\n", df[df.duplicated()].head())

Number of duplicate rows: 61
Sample duplicates:
       FiscalYear DecisionStatus DecisionDate EffectiveDate ExpirationDate  \
148         2007       Approved   11/27/2006    2006-11-27     2007-02-20   
334         2007       Approved   11/16/2006    2007-09-13     2007-06-25   
798         2007       Approved    3/26/2007    2007-03-01     2008-04-01   
1404        2007       Approved    3/26/2007    2007-03-01     2008-09-01   
1990        2007       Approved    10/5/2006    2006-10-01     2007-09-30   

     Brokered DealCancelled               Country      ProgramType PolicyType  \
148        No            No         United States  Working Capital    Unknown   
334        No            No                Brazil        Guarantee    Unknown   
798       Yes            No  Multiple - Countries        Insurance        ELC   
1404      Yes            No  Multiple - Countries        Insurance        ELC   
1990       No            No  Multiple - Countries        Insurance        ENB   

 

**2.4.2 Drop duplicate rows and verify**

In [17]:
# 3. Drop duplicate rows
df = df.drop_duplicates()
print("Number of duplicate rows after cleanup:", df.duplicated().sum())
print("Final shape of dataset:", df.shape)

Number of duplicate rows after cleanup: 0
Final shape of dataset: (51692, 24)


**We do the following steps to ensures each record is unique.**

- Checked for duplicate rows using `df.duplicated()`.  
- Previewed a few duplicates to confirm.  
- Dropped duplicates with `df.drop_duplicates()`.  
- Verified that no duplicates remain and noted the final dataset shape.
  
Even when no duplicates are detected here, including this step makes that duplicate handling was considered and confirms the dataset is unique.


**2.5 Correcting data types**

**2.5.1 Convert date columns to datetime**

In [18]:
date_cols = ['DecisionDate', 'EffectiveDate', 'ExpirationDate']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(df[date_cols].dtypes)

DecisionDate      datetime64[ns]
EffectiveDate     datetime64[ns]
ExpirationDate    datetime64[ns]
dtype: object


**2.5.2 Covert Numeric Columns into float datatypes**

In [19]:
numeric_cols = ['ApprovedAmount', 'DisbursedAmount', 'InterestRate']
for col in numeric_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(',', '', regex=False)
            .str.replace('%', '', regex=False)
            .astype(float)
        )

print(df[numeric_cols].dtypes)

ApprovedAmount     float64
DisbursedAmount    float64
InterestRate       float64
dtype: object


**2.5.3 Convert Categorical Columns**

In [20]:
#Define target categorical columns
categorical_cols = ['Country', 'ProgramType', 'DecisionStatus', 'Lender']

#Convert only existing columns to string
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

#Print dtypes of only existing columns
existing_categorical_cols = [col for col in categorical_cols if col in df.columns]
print(" Categorical column types:\n", df[existing_categorical_cols].dtypes)

#check all data types
print(" Final Data Types:\n", df.dtypes.head(12))


 Categorical column types:
 Country           object
ProgramType       object
DecisionStatus    object
dtype: object
 Final Data Types:
 FiscalYear                     int64
DecisionStatus                object
DecisionDate          datetime64[ns]
EffectiveDate         datetime64[ns]
ExpirationDate        datetime64[ns]
Brokered                      object
DealCancelled                 object
Country                       object
ProgramType                   object
PolicyType                    object
DecisionAuthority             object
ProductDescription            object
dtype: object


Here, we convert all data types which have wrong data types:
  - Dates - `datetime64[ns]`
  - Numeric - `float64`
  - Categorical - `object` (string)


**2.6 Creating derived columns**

**2.6.1 Fiscal Year from DecisionDate**
- Derived `FiscalYear` from `DecisionDate` using `.dt.year`.  
- Useful for year‑wise grouping and trend analysis.

In [21]:
if 'DecisionDate' in df.columns:
    df['FiscalYear'] = df['DecisionDate'].dt.year

print(df[['DecisionDate', 'FiscalYear']].head())

  DecisionDate  FiscalYear
0   2006-11-29        2006
1   2006-11-29        2006
2   2007-08-16        2007
3   2007-09-21        2007
4   2006-11-24        2006


**2.6.2 Loan Duration (in days)**
- Calculate `LoanDuration` is the difference between `ExpirationDate` and `EffectiveDate`.  
- Expressed in days for precise contract length analysis.


In [22]:
if 'EffectiveDate' in df.columns and 'ExpirationDate' in df.columns:
    df['LoanDuration'] = (df['ExpirationDate'] - df['EffectiveDate']).dt.days

print(df[['EffectiveDate', 'ExpirationDate', 'LoanDuration']].head())

  EffectiveDate ExpirationDate  LoanDuration
0    2006-11-29     2011-12-15          1842
1    2006-11-29     2016-12-15          3669
2    2007-08-16     2017-09-15          3683
3    2007-09-21     2008-07-31           314
4    2006-11-24     2007-01-01            38


**2.6.3 Approval Ratio**
- calculate `ApprovalRatio` as `DisbursedAmount / ApprovedAmount`.  
- It indicates how much of the approved loan was actually disbursed.

In [23]:
if 'DisbursedAmount' in df.columns and 'ApprovedAmount' in df.columns:
    df['ApprovalRatio'] = df['DisbursedAmount'] / df['ApprovedAmount']

print(df[['ApprovedAmount', 'DisbursedAmount', 'ApprovalRatio']].head())

   ApprovedAmount  DisbursedAmount  ApprovalRatio
0     24500000.00      24500000.00           1.00
1     50000000.00      50000000.00           1.00
2    136250000.00     136250000.00           1.00
3      2700000.00       2700000.00           1.00
4       233307.90        233307.90           1.00


**2.6.4 Interest Category**

- Classify loans into `InterestCategory` based on `InterestRate`.  
- Categories: Low (0–5%), Moderate (5–10%), High (10–20%), Very High (>20%).
- It Helps in risk profiling and comparative analysis.


In [24]:
if 'InterestRate' in df.columns:
    df['InterestCategory'] = pd.cut(
        df['InterestRate'],
        bins=[0, 5, 10, 20, 100],
        labels=['Low', 'Moderate', 'High', 'Very High']
    )

print(df[['InterestRate', 'InterestCategory']].head())

   InterestRate InterestCategory
0          3.13              Low
1          3.13              Low
2          3.13              Low
3          3.13              Low
4          3.13              Low


**2.7 Filtering or aggregating data**

**2.7.1 Filtering Rows**

***Example 1: Filter loans approved in 2024***

- Filtering loans where `FiscalYear = 2024`.  
- It is useful for analyzing approvals specific to a given year.


In [25]:
loans_2024 = df[df['FiscalYear'] == 2024]
print(loans_2024[['FiscalYear', 'ApprovedAmount', 'Country']].head())

       FiscalYear  ApprovedAmount        Country
49280        2024    156562500.00        Ukraine
49281        2024     22709850.00  United States
49282        2024     51803720.00  United States
49283        2024     19062863.00          Japan
49284        2024     98000000.00        Romania


***Example 2: Filter loans with InterestRate > 10%***

- Selected loans with `InterestRate > 10%`.  
- Helps identify higher‑risk or costlier loans.

In [26]:
high_interest_loans = df[df['InterestRate'] > 10]
print(high_interest_loans[['InterestRate', 'Country', 'ProgramType']].head())


Empty DataFrame
Columns: [InterestRate, Country, ProgramType]
Index: []


**2.7.2 Aggregating Data**

***Example 1: Total ApprovedAmount by FiscalYear***

- Grouping loans by `FiscalYear`.  
- Calculate total `ApprovedAmount` for each year using `.sum()`.  
- It is useful for year-wise trend analysis and budget planning.


In [27]:
# Group by FiscalYear and sum ApprovedAmount
approved_by_year = df.groupby('FiscalYear')['ApprovedAmount'].sum()
print(approved_by_year)

FiscalYear
2006    1141971723.67
2007   12828061681.33
2008   15521602468.39
2009   28069811228.51
2010   21795700310.39
2011   29664372264.79
2012   41722269785.00
2013   21996519923.90
2014   19451546609.31
2015   11042188599.44
2016    3932488951.43
2017    3416110928.69
2018    3142771239.61
2019    8044012433.86
2020    6768182834.10
2021    5272077898.61
2022    6004055654.53
2023    8372962082.45
2024    9043649031.64
2025    2388929183.51
Name: ApprovedAmount, dtype: float64


***Example 2: Average InterestRate by Country***

- Grouping loans by `Country`.  
- Calculate average `InterestRate` using `.mean()`.  
- It Helps to compare loan costs across regions.


In [28]:
avg_interest_by_country = df.groupby('Country')['InterestRate'].mean()
print(avg_interest_by_country)

Country
Algeria                    3.13
Angola                     4.08
Argentina                  3.12
Aruba                      3.13
Australia                  3.08
                           ... 
Uruguay                    3.13
Uzbekistan                 3.13
Vietnam                    3.13
Virgin Islands (British)   3.13
Zambia                     3.13
Name: InterestRate, Length: 152, dtype: float64


***Example 3: Count of Loans by ProgramType***

- Grouping loans by `ProgramType`.  
- Counting number of entries using `.size()` for total rows or `.count()` for non-null `DealNumber`.  
- This reveals program popularity and supports categorical analysis.

In [29]:
loan_count_by_program = df.groupby('ProgramType').size()
print(loan_count_by_program)

ProgramType
Guarantee           2822
Insurance          42348
Loan                 367
Working Capital     6155
dtype: int64


**2.8 Final Data Quality Check**

In [35]:
print("FINAL DATA QUALITY CHECK")

# 1. Dataset dimensions
print("\n1. Dataset Dimensions:")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")

# 2. Missing values check
print("\n2. Missing Values:")
total_missing = df.isnull().sum().sum()
print(f"  Total missing values: {total_missing}")

# 3. Data types summary
print("\n3. Data Types Summary:")
print(f"  Numeric columns: {df.select_dtypes(include=['float64','int64']).shape[1]}")
print(f"  Categorical columns: {df.select_dtypes(include=['object']).shape[1]}")

# 4. Column Data Types
print("\n4. Column Data Types:")
for col in df.columns:
    print(f"  {col:25s}: {str(df[col].dtype):10s}")

print("\n Data Preprocessing Complete")

FINAL DATA QUALITY CHECK

1. Dataset Dimensions:
  Rows: 51,692
  Columns: 27

2. Missing Values:
  Total missing values: 823

3. Data Types Summary:
  Numeric columns: 10
  Categorical columns: 12

4. Column Data Types:
  FiscalYear               : int32     
  DecisionStatus           : object    
  DecisionDate             : datetime64[ns]
  EffectiveDate            : datetime64[ns]
  ExpirationDate           : datetime64[ns]
  Brokered                 : object    
  DealCancelled            : object    
  Country                  : object    
  ProgramType              : object    
  PolicyType               : object    
  DecisionAuthority        : object    
  ProductDescription       : object    
  Term                     : object    
  Exporter                 : object    
  ExporterState            : object    
  PSOR                     : object    
  ApprovedAmount           : float64   
  DisbursedAmount          : float64   
  UndisbursedExposure      : float64   
  Outst

We verify dataset dimensions, missing values, and column data types.  

The dataset is now clean, consistent, and ready for exploratory analysis.

**2.9 Exporting Cleaned Dataset**

In [36]:
import os

# Define export path
export_path = "../data/FY25_Q3_Cleaned.csv"

# Create folder if it doesn't exist
os.makedirs(os.path.dirname(export_path), exist_ok=True)

# Export cleaned dataset
df.to_csv(export_path, index=False)

print("✓ Dataset exported successfully!")
print(f" Location: {export_path}")
print(f" Rows: {len(df)}")
print(f" Columns: {len(df.columns)}")

✓ Dataset exported successfully!
 Location: ../data/FY25_Q3_Cleaned.csv
 Rows: 51692
 Columns: 27


We save the cleaned dataset as a CSV file for use in the next step (Exploratory Data Analysis).

This will ensure the reproducibility and separates raw data from processed data.

***Reviewing the Steps Completed Above***

In the first stage, the dataset was imported using pd.read_csv() and examined with .head(), .info(), and .describe() to understand its structure, column types, missing values, and basic statistics. This provided clarity on key fields such as dates, amounts, and categories, and confirmed the dataset’s shape for planning further steps. In the second stage, Data Pre-processing, the data was cleaned and standardized by renaming columns, dropping irrelevant fields, handling missing values, and removing duplicates. Data types were corrected by converting dates to datetime, numeric fields to float, and categorical fields to strings. Derived columns such as FiscalYear, LoanDurationDays, ApprovalRatio, and InterestCategory were created to enrich analysis. Finally, filtering and aggregation were applied to extract insights, including year-wise totals, average interest rates by country, and loan counts by program type. Together, these stages ensured the dataset was reliable, structured, and ready for visualization and deeper business insights.